In [4]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from mpl_toolkits.basemap import Basemap

In [9]:
bounds_min_lat = 32.0
bounds_max_lat = 36.0
bounds_min_lon = -120.0
bounds_max_lon = -116.0

bounds_lat = slice(bounds_min_lat, bounds_max_lat)
bounds_lon = slice(bounds_min_lon, bounds_max_lon)

In [10]:
# Open zarr datasets

l3_pre_qf = xr.open_zarr('data/zarr/la_sams_sample_pre_qf.zarr', consolidated=True).sel(latitude=bounds_lat, longitude=bounds_lon)
l3_post_qf = xr.open_zarr('data/zarr/la_sams_sample_post_qf.zarr', consolidated=True).sel(latitude=bounds_lat, longitude=bounds_lon)

l3_post_qf

<xarray.Dataset>
Dimensions:           (latitude: 160, longitude: 160, time: 8)
Coordinates:
  * latitude          (latitude) float32 32.02 32.04 32.07 ... 35.94 35.97 35.99
  * longitude         (longitude) float32 -120.0 -120.0 -119.9 ... -116.0 -116.0
  * time              (time) datetime64[ns] 2020-03-03T08:00:00 ... 2022-10-2...
Data variables:
    xco2              (time, longitude, latitude) float64 dask.array<chunksize=(5, 100, 120), meta=np.ndarray>
    xco2_apriori      (time, longitude, latitude) float64 dask.array<chunksize=(5, 100, 120), meta=np.ndarray>
    xco2_uncertainty  (time, longitude, latitude) float64 dask.array<chunksize=(5, 100, 120), meta=np.ndarray>

In [8]:
# ds = xr.open_mfdataset('data/netcdf/*.nc4')
# ds_s = xr.open_mfdataset('data/netcdf/*.nc4', group='Sounding', concat_dim='sounding_id', combine='nested')

# ds
# ds_s

granules = [
    "data/netcdf/oco3_LtCO2_200303_B10400Br_220318000013s.nc4",
    "data/netcdf/oco3_LtCO2_200505_B10400Br_220318001036s.nc4",
    "data/netcdf/oco3_LtCO2_200527_B10400Br_220318001255s.nc4",
    "data/netcdf/oco3_LtCO2_200814_B10400Br_220318002549s.nc4",
    "data/netcdf/oco3_LtCO2_210325_B10400Br_220318010127s.nc4",
    "data/netcdf/oco3_LtCO2_220218_B10400Br_220505141844s.nc4",
    "data/netcdf/oco3_LtCO2_220813_B10400Br_221010202453s.nc4",
    "data/netcdf/oco3_LtCO2_221028_B10400Br_221205203441s.nc4",
]



<xarray.Dataset>
Dimensions:               (sounding_id: 1183178)
Dimensions without coordinates: sounding_id
Data variables: (12/21)
    solar_azimuth_angle   (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    sensor_azimuth_angle  (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    pma_elevation_angle   (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    pma_azimuth_angle     (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    polarization_angle    (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    att_data_source       (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    ...                    ...
    target_id             (sounding_id) object dask.array<chunksize=(252017,), meta=np.ndarray>
    target_name           (sounding_id) object dask.array<chunksize=(252017,), meta=np.ndarray>
    land_water_indicator  (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    altitude              (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    altitude_stddev       (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>
    zlo_wco2              (sounding_id) float32 dask.array<chunksize=(252017,), meta=np.ndarray>